In [ ]:
import os
from pathlib import Path
os.environ['AOC_SESSION'] = Path('session.txt').read_text().strip()

In [ ]:
from aocd.models import Puzzle

# Day 1

In [ ]:
puzzle = Puzzle(2021, 1)

In [ ]:
data = list(map(int, puzzle.input_data.split()))

In [ ]:
data

## Part 1 

In [ ]:
res_a = sum(e1 < e2 for e1, e2 in zip(data, data[1:]))

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [ ]:
res_b = sum(e1 < e4 for e1, e4 in zip(data, data[3:]))

In [ ]:
puzzle.answer_b = res_b

# Day 2

In [ ]:
puzzle = Puzzle(2021, 2)

In [ ]:
data = list(map(lambda s: s.split(), puzzle.input_data.split("\n")))

In [ ]:
data

## Part 1 

In [ ]:
from collections import Counter
c = Counter()
for x, y in data:
    c.update({x:int(y)})


In [ ]:
c

In [ ]:
res_a = (c["down"] - c["up"]) * c["forward"]

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [ ]:
aim = 0
horizontal = 0
depth = 0
for x, y in data:
    y = int(y)
    if x == "down":
        aim += y
    elif x == "up":
        aim -= y
    else:
        horizontal += y
        depth += aim*y

In [ ]:
res_b = horizontal * depth

In [ ]:
puzzle.answer_b = res_b

# Day 3

In [ ]:
puzzle = Puzzle(2021, 3)

In [ ]:
data = puzzle.input_data

In [ ]:
data

## Part 1 

In [ ]:
res_a = ...    

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [ ]:
res_b = ...

In [ ]:
puzzle.answer_b = res_b

# Day 4

In [ ]:
puzzle = Puzzle(2021, 4)

In [ ]:
data = puzzle.input_data

In [ ]:
data

## Part 1 

In [ ]:
res_a = ...    

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [ ]:
res_b = ...

In [ ]:
puzzle.answer_b = res_b

# Day 5

In [ ]:
puzzle = Puzzle(2021, 5)

In [ ]:
data = puzzle.input_data

In [ ]:
data

## Part 1 

In [ ]:
res_a = ...    

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [ ]:
res_b = ...

In [ ]:
puzzle.answer_b = res_b

# Day 6

In [ ]:
puzzle = Puzzle(2021, 6)

In [ ]:
data = puzzle.input_data

In [ ]:
data

## Part 1 

In [ ]:
res_a = ...    

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [ ]:
res_b = ...

In [ ]:
puzzle.answer_b = res_b